In [6]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
import xgboost
from sklearn.model_selection import train_test_split

# train and test datasets
train_data = pd.read_csv("files/aapl_5m_train.csv")
test_data = pd.read_csv("files/aapl_5m_validation.csv")


# logistic regression
df = pd.DataFrame(train_data)
df = df[["Datetime","Close"]].set_index("Datetime")

df1 = df.shift(periods=-1)
df2 = df1.shift(periods=-1)
df3 = df2.shift(periods=-1)
df4 = df3.shift(periods=-1)

a = pd.DataFrame({})

a['X1'] = df
a['X2'] = df1
a['X3'] = df2
a['X4'] = df3
a['X5'] = df4

a["Y"] = a["X5"].shift(-5) > a["X5"]

a = a.dropna()

# variables
x = a.drop('Y', axis = 1)
y = a['Y']

y_int = train_y.astype(int)

# convert binary
array_binario = np.where(y_int == 1, 1, 0)

class LogisticRegression:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        # Initialize weights and bias
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0

        # Gradient descent
        for _ in range(self.num_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(linear_model)

            # Compute gradients
            dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / num_samples) * np.sum(y_pred - y)

            # Update weights and bias
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = self.sigmoid(linear_model)
        y_pred_class = [1 if i > 0.5 else 0 for i in y_pred]
        return y_pred_class
    
log = LogisticRegression()
log.fit(train_x, array_binario)
y_p = log.predict(train_x)

# evaluation of the model
confusion_matrix_1 = np.zeros((2, 2))

for i in range(len(array_binario)):
    true_label = int(array_binario[i])  # Convertir a entero
    predicted_label = int(y_p[i])  # Convertir a entero
    confusion_matrix_1[true_label][predicted_label] += 1

print("Matriz de Confusión:")
print(confusion_matrix_1)

TP = confusion_matrix_1[1, 1]  # Verdaderos Positivos
TN = confusion_matrix_1[0, 0]  # Verdaderos Negativos
FP = confusion_matrix_1[0, 1]  # Falsos Positivos
FN = confusion_matrix_1[1, 0]  # Falsos Negativos

total_samples = np.sum(confusion_matrix_1)

accuracy = (TP + TN) / total_samples
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print("accuracy, precision, recall del LR: ",accuracy, precision, recall)


# svc
rets = train_data.loc[:,['Close']].pct_change().dropna()

x = pd.DataFrame()
x['X0'] = rets.Close
x['X1'] = rets.Close.shift(-1)
x['X2'] = rets.Close.shift(-2)
x['X3'] = rets.Close.shift(-3)
x['X4'] = rets.Close.shift(-4)
x['X5'] = rets.Close.shift(-5)
x = x.dropna()

y = (train_data.Close.shift(-5) * 1.005 < train_data.Close.shift(-15))[:-6]
assert len(x) == len(y)

model = SVC(random_state =42)
model.fit(x,y)

y_hat = model.predict(x)
confusion_matrix(y,y_hat)

# Prueba con datos train
model = SVC(random_state=42)
model.fit(train_x, train_y)


# xgboost
boosted_model = xgboost.XGBClassifier()
boosted_model.fit(x, y)

y_hat_boost = boosted_model.predict(x)

confusion_matrix(y,y_hat_boost) #xgboost
confusion_matrix(y,y_hat) #svc

accuracy_score(y,y_hat_boost)

def optimize_params(x_:np.array) -> float:
    gamma, reg_alpha = x_ #unpack parameters
    n_estimators = 1

    # X Y
    model_ = xgboost.XGBClassifier(n_estimators=n_estimators,
                                  gamma=gamma,
                                  reg_alpha=reg_alpha,
                                  reg_lambda=reg_alpha)
    model_.fit(x,y)
    y_pred = model_.predict(x)
    acc = accuracy_score(y, y_pred)

    return -acc

bnds = ((1e-3,10), (1e-4,10))
x0 = [1, 1e-3]

res = minimize(optimize_params, bounds=bnds, x0=x0, method="Nelder-Mead", tol=1e-6)
optimal_params = res.x

print("Hiperparámetros óptimos:", optimal_params)

opt_model = xgboost.XGBClassifier(n_estimators=10,
                                  gamma=1.05e+01,
                                  reg_alpha=1.00e-03,
                                  reg_lambda=1.00e-03)

#Prueba de evaluacion de modelo
boosted_model = xgboost.XGBClassifier()
boosted_model.fit(train_x, train_y)

y_hat_boost = boosted_model.predict(X_test)
confusion_matrix_xgb = confusion_matrix(y_test, y_hat_boost)
accuracy_xgb = accuracy_score(y_test, y_hat_boost)

print("Matriz de confusión XGBoost:\n", confusion_matrix_xgb)
print("Precisión del modelo XGBoost: {:.2f}%".format(accuracy_xgb * 100))


# Combinations
n = list(range(1, 2**7))
combinations = list(map(lambda x: [int(bit) for bit in f"{x:07b}"], n))

# REVISAR ESTE CODIGO (ALGO MAL )
comission = .025
stop_loss = .035
take_profit = .035
cash = 1000000
positions = []
operations = []
trading_data = train_d
sma_period = 10
rsi_period = 14

class Backtesting:
    def __init__(self, trading_data, comission, stop_loss, take_profit):
        self.trading_data = trading_data
        self.comission = comission
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.sma_period = sma_period
        self.rsi_period = rsi_period
        self.cash = 1000000  # Initial cash balance
        self.positions = []

    class Order:
        def __init__(self, timestamp, bought_at, stop_loss,
                     take_profit, order_type, sold_at=None,
                     is_active=True):
            self.timestamp = timestamp
            self.bought_at = bought_at
            self.sold_at = sold_at
            self.stop_loss = stop_loss
            self.take_profit = take_profit
            self.order_type = order_type
            self.is_active = is_active

    def backtest(self):
        for i, row in self.trading_data.iterrows():
            # Close positions
            price = row['Close']
            for position in self.positions:
                if position.is_active:
                    if price <= position.stop_loss:
                        self.cash += price * (1 - self.comission)
                        position.is_active = False
                        position.sold_at = price
                    elif price >= position.take_profit:
                        self.cash += price * (1 - self.comission)
                        position.is_active = False
                        position.sold_at = price

            # Buy SMA
            if row['SMA_Buy_Signal']:
                if self.cash >= row['Close'] * (1 + self.comission):
                    self.cash -= row['Close'] * (1 + self.comission)
                    order = self.Order(timestamp=row['Timestamp'],
                                       bought_at=row['Close'],
                                       stop_loss=row['Close'] * (1 - self.stop_loss),
                                       take_profit=row['Close'] * (1 + self.take_profit),
                                       order_type="LONG")
                    self.positions.append(order)

            # Buy MACD
            if row['MACD_Buy_Signal']:
                if self.cash >= row['Close'] * (1 + self.comission):
                    self.cash -= row['Close'] * (1 + self.comission)
                    order = self.Order(timestamp=row['Timestamp'],
                                       bought_at=row['Close'],
                                       stop_loss=row['Close'] * (1 - self.stop_loss),
                                       take_profit=row['Close'] * (1 + self.take_profit),
                                       order_type="LONG")
                    self.positions.append(order)

            # Buy RSI
            if row['RSI_Buy_Signal']:
                if self.cash >= row['Close'] * (1 + self.comission):
                    self.cash -= row['Close'] * (1 + self.comission)
                    order = self.Order(timestamp=row['Timestamp'],
                                       bought_at=row['Close'],
                                       stop_loss=row['Close'] * (1 - self.stop_loss),
                                       take_profit=row['Close'] * (1 + self.take_profit),
                                       order_type="LONG")
                    self.positions.append(order)

            # Sell SMA
            if row['SMA_Sell_Signal']:
                for position in self.positions:
                    if position.is_active and position.order_type == "LONG":
                        if row['Close'] * (1 - self.comission) >= position.take_profit:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
                        elif row['Close'] * (1 + self.comission) <= position.stop_loss:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']

            # Sell MACD
            if row['MACD_Sell_Signal']:
                for position in self.positions:
                    if position.is_active and position.order_type == "LONG":
                        if row['Close'] * (1 - self.comission) >= position.take_profit:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
                        elif row['Close'] * (1 + self.comission) <= position.stop_loss:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']

            # Sell RSI
            if row['RSI_Sell_Signal']:
                for position in self.positions:
                    if position.is_active and position.order_type == "LONG":
                        if row['Close'] * (1 - self.comission) >= position.take_profit:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
                        elif row['Close'] * (1 + self.comission) <= position.stop_loss:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
        print("Final Cash Balance:", self.cash)
        #x= self.cash
        print("Open Positions:")
        for position in self.positions:
            print(f"Timestamp: {position.timestamp}, Bought at: {position.bought_at}, Sold at: {position.sold_at}")


    def optimize_strategy(self, parameters):
        sl, tp, *strat_args = parameters
        self.stop_loss = sl
        self.take_profit = tp
        # Set other strategy arguments here if needed

        # Perform backtesting with the updated parameters
        self.backtest()

# Create an instance of the Backtesting class
backtester = Backtesting(trading_data, comission, stop_loss, take_profit)

# Define the optimization function
def minimize_port(params):
    backtester.optimize_strategy(params)
    return -backtester.cash  # You want to maximize the final cash balance

# Parameter optimization
parameter_bounds = [(0.01, 0.05),
                    (0.01, 0.05),
                    (1, 50),
                    (51, 220)]

# Initial guess for parameters
initial_guess = [0.03, 0.03, 25, 135]  # Adjust these values as needed

# Perform optimization
result = minimize(minimize_port, initial_guess, bounds=parameter_bounds)

# Retrieve optimized parameters
optimal_params = result.x

# Print the optimal parameters and final cash balance
print("Optimal Parameters:", optimal_params)
print("Final Cash Balance:", -result.fun)

Matriz de Confusión:
[[9808.    0.]
 [9975.    0.]]


C:\Users\marmr\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:104: RuntimeWarning: invalid value encountered in double_scalars


Hiperparámetros óptimos: [1.    0.001]


NameError: name 'X_test' is not defined